In [29]:
import pandas as pd

data=pd.read_csv('D:/Bahar/Doctora/ML Projects/New Fake News Detection/Dataset/train.csv')
print(data.head())


   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  


In [30]:
tags = data["label"]
texts = data["text"]
print(texts[0])

House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) 
With apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. 
As we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emails 

In [32]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords

def process_sms(vocab):
    for t in texts:
        tokens = clean_txt(t)
        vocab.update(tokens)

def clean_txt(txt):
    # split into tokens by white space
    txt = str(txt)
    tokens = txt.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

vocab = Counter()
process_sms(vocab)
print(len(vocab))
print(vocab.most_common(50))

221093
[('The', 90573), ('said', 79824), ('Mr', 66039), ('Trump', 43217), ('would', 36693), ('people', 32311), ('one', 31575), ('In', 26619), ('like', 24554), ('also', 24421), ('But', 23160), ('Clinton', 22010), ('He', 20298), ('time', 19076), ('could', 18829), ('years', 17208), ('It', 17184), ('many', 16596), ('new', 16492), ('even', 15966), ('two', 15579), ('And', 15536), ('American', 15327), ('first', 15189), ('United', 14689), ('government', 14338), ('US', 14002), ('This', 13614), ('last', 13370), ('Hillary', 13073), ('New', 12963), ('year', 12790), ('get', 12753), ('Ms', 12568), ('President', 12485), ('Obama', 12305), ('States', 12153), ('going', 12122), ('campaign', 11902), ('way', 11709), ('made', 11700), ('make', 11595), ('know', 11444), ('much', 11127), ('think', 11123), ('back', 11003), ('election', 10738), ('world', 10717), ('say', 10711), ('state', 10634)]


In [33]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))
vocab = set(tokens)

120536


In [34]:
# load embedding as a dict
def load_embedding(filename):
    file = open(filename,'r',encoding="utf-8")
    lines = file.readlines()
    file.close()
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding

In [35]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    #vocab_size = len(vocab) + 1
    vocab_size = 1000
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        vector = embedding.get(word)
        if vector is not None and i < 1000:
            weight_matrix[i] = vector
    return weight_matrix

In [37]:
def process_news():
    documents = list()
    for text in texts:
        tokens = clean_text(text, vocab)
        documents.append(tokens)
    return documents

def clean_text(text, vocab):
    # split into tokens by white space
    tokens = str(text).split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens

vocab = set(tokens)
train_texts = process_news()

print(train_texts[0])

House Dem Aide We Even See Letter Until Jason Chaffetz Tweeted It By Darrell Lucus October Subscribe Jason Chaffetz stump American Fork Utah image courtesy Michael available Creative license With apologies Keith Olbermann doubt Worst Person The World Director James Comey But according House Democratic aide looks like also know person well It turns Comey sent nowinfamous letter announcing FBI looking emails may related Hillary email server ranking Democrats relevant committees hear Comey They found via tweet one Republican committee chairmen As know Comey notified Republican chairmen Democratic ranking members House Intelligence Judiciary Oversight committees agency reviewing emails recently discovered order see contained classified information Not long letter went Oversight Committee Chairman Jason Chaffetz set political world ablaze tweet FBI Dir informed The FBI learned existence emails appear pertinent investigation Case reopened Jason Chaffetz jasoninthehouse October Of course know

In [38]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import merge, Dense, LSTM, Dropout
from keras.layers import Flatten, Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_texts)
# sequence encode
encoded_texts = tokenizer.texts_to_sequences(train_texts)
# pad sequences

max_length = max([len(s.split()) for s in train_texts])
All_news = pad_sequences(encoded_texts, maxlen=1000, padding='post')

In [39]:
#Splitting test and train sets
Xtrain = All_news[:16640] #80% of all news
Xtest = All_news[16640:]

ytrain = tags[:16640]
ytest = tags[16640:]

from keras.utils import to_categorical
ytrain = to_categorical(ytrain)
ytest = to_categorical(ytest)


In [40]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

100706


In [41]:
from numpy import asarray
from numpy import zeros

# define vocabulary size (largest integer value)
vocab_size = 1000
# load embedding from file
raw_embedding = load_embedding('glove.6B.100d.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)


In [27]:
from keras import Input
from keras import Model
from keras.layers import concatenate

input_x = Input(shape=(None,))
embed=Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=1000, trainable=False)(input_x)

cnn1=Conv1D(nb_filter=128, filter_length=3,border_mode='same',activation='relu')(embed)
max_pool1 = MaxPooling1D(pool_size=5)(cnn1)

cnn2=Conv1D(nb_filter=128, filter_length=4,border_mode='same',activation='relu')(embed)
max_pool2 = MaxPooling1D(pool_size=5)(cnn2)

cnn3=Conv1D(nb_filter=128, filter_length=5,border_mode='same',activation='relu')(embed)
max_pool3 = MaxPooling1D(pool_size=5)(cnn3)

concatenated = concatenate([max_pool1,max_pool2, max_pool3],axis = -1)

cnn_l1 = Conv1D(filters=128, kernel_size=5, activation='relu')(concatenated)
max1 = MaxPooling1D(pool_size=5)(cnn_l1)

cnn_l2 = Conv1D(filters=128, kernel_size=5, activation='relu')(max1)
max2 = MaxPooling1D(pool_size=5)(cnn_l2)

flat = Flatten()(max2)

den1=Dense(128, activation='softmax')(flat)
den1 = Dropout(0.2)(den1)
den2=Dense(2,activation='softmax')(den1)

FNDNet = Model(input_x, output=den2)
FNDNet.compile(loss='categorical_crossentropy', optimizer='adadelta',metrics=['accuracy'])
FNDNet.summary()


C:\Users\sattar\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3, padding="same")`
  
C:\Users\sattar\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4, padding="same")`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\sattar\anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=5, padding="same")`
  


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    100000      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1000, 128)    38528       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 1000, 128)    51328       embedding_1[0][0]                
____________________________________________________________________________________________

C:\Users\sattar\anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("de...)`


In [28]:
FNDNet.fit(Xtrain, ytrain, epochs=5)
# evaluate
loss, acc = FNDNet.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))


Epoch 1/5


InvalidArgumentError: indices[1,0] = 1347 is not in [0, 1000)
	 [[{{node embedding_1/embedding_lookup}}]]